In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt

%matplotlib inline
mpl.style.use('bmh')

In [183]:
import numpy as np

import torch
import math
import random

from tqdm import tqdm_notebook
from boltons.iterutils import pairwise
from scipy.stats import kendalltau
from itertools import permutations

from sent_order.models.kt_regression import SentenceEncoder, Regressor, Corpus
from sent_order.perms import sample_uniform_perms

In [3]:
sent_encoder = torch.load(
    '../../data/models/new/kt-reg/sent_encoder.68.bin',
    map_location={'cuda:0': 'cpu'},
)

In [ ]:
regressor = torch.load(
    '../../data/models/new/kt-reg/regressor.68.bin',
    map_location={'cuda:0': 'cpu'},
)

In [5]:
train = Corpus('../../data/train.json/', 10000)

100%|██████████| 10000/10000 [00:01<00:00, 8161.12it/s]


In [211]:
grafs = [g for g in train.grafs if len(g.sentences) == 10]

In [212]:
len(grafs)

313

In [213]:
graf = grafs[100]

In [214]:
sents = sent_encoder(graf.sentence_variables())

In [215]:
regressor(sents.unsqueeze(0)).data[0]

0.029865384101867676

In [207]:
[' '.join(s.tokens) for s in graf.sentences]

['Generalized belief propagation ( GBP ) has proven to be a promising technique for approximate inference tasks in AI and machine learning .',
 'However , the choice of a good set of clusters to be used in GBP has remained more of an art then a science until this day .',
 'This paper proposes a sequential approach to adding new clusters of nodes and their interactions ( i.e."regions " ) to the approximation .',
 'We first review and analyze the recently introduced region graphs and find that three kinds of operations ( " split " , " merge " and " death " ) leave the free energy and ( under some conditions ) the fixed points of GBP invariant .',
 'This leads to the notion of " weakly irreducible " regions as the natural candidates to be added to the approximation .',
 'Computational complexity of the GBP algorithm is controlled by restricting attention to regions with small " region - width " .',
 'Combining the above with an efficient ( i.e.local in the graph ) measure to predict the i

In [200]:
predict(graf)

(0, 1, 2)

In [201]:
def predict(graf):
    
    sents = sent_encoder(graf.sentence_variables())
    
    perms = list(permutations(range(len(sents))))
    
    x = torch.stack([
        sents[torch.LongTensor(perm)]
        for perm in perms
    ])
    
    scores = regressor(x)
    
    return perms[torch.min(scores, 0)[1].data[0]]

In [202]:
kts = []
for g in tqdm_notebook(grafs[:100]):
    pred = predict(g)
    kt = kendalltau(pred, range(len(pred))).correlation
    kts.append(kt)

HBox(children=(IntProgress(value=0), HTML(value='')))

Exception in thread Thread-13:
Traceback (most recent call last):
  File "/usr/local/Cellar/python3/3.6.2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/Users/dclure/Projects/plot-ordering/env/lib/python3.6/site-packages/tqdm/_tqdm.py", line 144, in run
    for instance in self.tqdm_cls._instances:
  File "/usr/local/bin/../Cellar/python3/3.6.2/bin/../Frameworks/Python.framework/Versions/3.6/lib/python3.6/_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration



In [203]:
np.mean(kts)

0.91333333333333333